# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

Need to install PyGithub via uv and and githubtoken in env file to make it work

In [1]:
# imports
import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from github import Github
from collections import defaultdict

In [2]:
# constants

MODEL_GEMINI = 'gemini-flash-latest'
MODEL_LLAMA = 'llama3.2'

In [3]:
# set up environment
load_dotenv(override=True)
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
google_api_key = os.getenv("GOOGLE_API_KEY")
github_api_token=os.getenv("GITHUB_TOKEN")

g = Github(github_api_token)

MODEL = 'gemini-flash-latest'
openai = OpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)

C:\Users\piyut\AppData\Local\Temp\ipykernel_11168\839515879.py:7: DeprecationWarning: Argument login_or_token is deprecated, please use auth=github.Auth.Token(...) instead
  g = Github(github_api_token)


In [4]:
def get_user_profile(username):
    user = g.get_user(username)
    return {
        "public_repos": user.public_repos,
        "followers": user.followers,
        "created_at": user.created_at.isoformat(),
    }


def get_repositories(username):
    user = g.get_user(username)
    return list(user.get_repos())  # handles pagination automatically


def aggregate_languages(repos):
    language_totals = defaultdict(int)

    for repo in repos:
        languages = repo.get_languages()  # returns dict like {"Python": 12345}
        for lang, bytes_of_code in languages.items():
            language_totals[lang] += bytes_of_code

    return dict(language_totals)


def build_summary(username):
    profile = get_user_profile(username)
    repos = get_repositories(username)
    languages = aggregate_languages(repos)

    summary = {
        "username": username,
        "public_repos": profile["public_repos"],
        "followers": profile["followers"],
        "account_age": profile["created_at"],
        "languages_used": languages,
        "total_repos_analyzed": len(repos),
    }

    return summary

In [5]:
# here is the question; type over this to ask something new
def extract_key_signals(summary):

    prompt = f"""
You are a technical signal extractor.

Given this GitHub summary, remove fluff and extract ONLY:

- Top primary languages (as percentages)
- Activity level (low / medium / high)
- Estimated experience level (beginner / intermediate / advanced)
- Development focus (frontend / backend / fullstack / ML / DevOps / etc.)
- Project depth (many small projects vs few complex ones)

Return ONLY valid JSON.

Data:
{json.dumps(summary, indent=2)}
"""

    response = openai.chat.completions.create(
        model=MODEL,
        response_format={"type": "json_object"},
        messages=[
            {"role": "system", "content": "You extract structured engineering signals."},
            {"role": "user", "content": prompt}
        ]
    )

    return json.loads(response.choices[0].message.content)



In [6]:
def generate_review(clean_profile):

    prompt = f"""
You are a senior software engineering mentor.

Based on this structured developer profile:

{json.dumps(clean_profile, indent=2)}

Provide:

1. Strengths
2. Skill gaps
3. What to study next
4. 3 concrete project ideas
5. Career advice

Be practical and actionable.
"""

    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "You are a senior engineering mentor."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content



In [7]:
# Get gpt-4o-mini to answer, with streaming
def fetchReview(username):
    raw_summary = build_summary(username)
    clean_profile = extract_key_signals(raw_summary)
    review = generate_review(clean_profile)
    return review

In [9]:
print(fetchReview('ed-donner'))

Hello. I’ve reviewed your profile. You are clearly a specialist with deep technical expertise in the **ML/AI/Data Science** domain. Your high activity level and focus on "few complex projects" suggest you are likely tackling heavy-lifting problems—perhaps model architecture, fine-tuning, or complex data pipelines.

However, your language distribution (92% Jupyter Notebook) reveals a classic "Researcher’s Trap." You are exceptional at the "Lab" phase but likely lack the "Factory" phase—turning those experiments into scalable, maintainable products.

Here is my assessment and roadmap for your next level.

---

### 1. Strengths
*   **Experimental Rigor:** You are likely highly proficient at the iterative process of data exploration, feature engineering, and model validation.
*   **Domain Depth:** Your focus on "few complex projects" rather than many small ones suggests you can handle high-dimensional problems and deep technical debt within a model's logic.
*   **Rapid Prototyping:** You c

In [ ]:
# Get Llama 3.2 to answer